## Simple recommendation system based on data from 1C exported dataset
#### Eventually the source dataset can be any kind of data with two features: 
1. Timestamp (date/time) of sale  
2. Item name/Art.  
Assuming that all the goods in check/invoice/ etc. were added as a new record in Accountant Database with the same timestamp/datetime record.
We can track the dependencies of these goods to meet each other in different sales.  
Here i've got the 1C exported dataset with various features, but i need only two of them (timestamp and art.)   

|Date Time|Art.|
|---|---|
|31.01.2020 10:21:47|а Ge 2530/75 напр.750мм для раздвижн. столов-"книж|
|31.01.2020 10:21:47|а  Buoni Подъемно-поворотн. м-зм цельной вставки|
|...|...|

In [2]:
# initial imports
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import glob
import datetime
from collections import Counter

### Initializing catalog dictionary, where we will collect all the items names as keys and Collections of similar goods as values:

In [4]:
### main catalog initiate
catalog = {}


### date parser for 1C
dateparse = lambda x: pd.datetime.strptime(x, "%d.%m.%Y %H:%M:%S")


def adopttocal(art):
    #########################################################################
    # function builds dictionary of Counters.                               #
    # key is the item and the value is counter in which                     #
    # it counts all the occurences with other items                         #
    # then, getting the item(art, name, or whatever) by key                 #
    # you will get the Counter (sorted by number of occurences eventually)  #
    # with all the recommended items                                        #
    # NOTE: to collect this dict it requires the big datasets               #
    #########################################################################
    for item in art:
        if item in catalog.keys():
            temp = catalog[item]
            for item_2 in art:
                if item_2 != item: # exclude self counting
                    temp[item_2] += 1
        else:
            temp = Counter()
            for item_2 in art:
                if item_2 != item: # exclude self counting
                    temp[item_2] += 1
            catalog[item] = temp

In [7]:
# importing sales excel files and gluing them together
# converting datetime data to unix-type timestamp as integer
files = glob.glob("datasets/sl*.xlsx")
frames = []
for f in files:
    df = pd.read_excel(
        f,
        usecols=[1, 2],
        skiprows=7,
        index_col=0,
        parse_dates=True,
        date_parser=dateparse,
    )
    df.columns = ['art']
    df.index.name = 'date'
    frames.append(df)
sales = pd.concat(frames, axis=0, sort=False)
sales['timestamp'] = sales.index.astype(int) / 10**9
sales.head()

,art,timestamp
date,,
2020-01-31 11:53:21,а Фрог-N45 мех-м с автоматическим подъемом вст...,1.580472e+09
2020-01-31 11:53:21,а 1280(D+H)FrZшкант металлический d=8мм,1.580472e+09
2020-01-31 11:53:21,а CY-83N -Замок кнопочный,1.580472e+09
2020-01-31 11:42:08,а K 0401 (Лизард)механизм (R+L) подъема вставки,1.580471e+09
2020-01-31 11:42:08,а IN 35/750/540 SL м-зм внут. креп.д/стол 750...,1.580471e+09


## The dictionary building part.
### Making the dataset of unique timestamps (sales) with list of each item within this sale.

In [9]:
sales.groupby(by='timestamp')['art'].apply(list).map(adopttocal)

# you can fetch by item name, but here we are fetching just for example by number of key
# and for example first 5 most valuable items
art_number = 5

print('\nTop {} recommended items for '.format(str(art_number)), list(catalog.keys())[art_number], ':')
# print('\n', catalog[list(catalog.keys())[5]].most_common(5))
for art, _ in catalog[list(catalog.keys())[art_number]].most_common(5):
    print(art)


Top 5 recommended items for  a Ge 655/110 м-зм д/столов, 110см,раздв.892мм :
a Ge 655/70 м-зм д/столов, 708мм,раздв.492мм
а 0135 замок для стола L=35мм
a Ge 655/55 м-зм д/столов, 558мм,раздв.342мм
a Ge 655/80 м-зм д/столов, 808мм,раздв.592мм
a Ge 655/90 м-зм д/столов, 908мм,раздв.692мм
